# t_50 plot

In [ ]:

import h5py
import numpy as np
with h5py.File('data/example_sleap_output.analysis.h5', 'r') as f:
    occupancy_matrix_sleap = f['track_occupancy'][:]
    
with h5py.File('data/example_naps_output.analysis.h5', 'r') as f:
    occupancy_matrix_naps = f['track_occupancy'][:]

aruco_only =  np.load("data/example_aruco_output.npy")
aruco_only = aruco_only[0:72000,0,0,:]
occupancy_matrix_aruco = ~np.isnan(aruco_only)

In [ ]:
%matplotlib inline

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

mpl.rcParams["figure.facecolor"] = "w"
mpl.rcParams["figure.dpi"] = 150
mpl.rcParams["savefig.dpi"] = 600
mpl.rcParams["savefig.transparent"] = True
mpl.rcParams["font.size"] = 15
mpl.rcParams["font.family"] = "sans-serif"
mpl.rcParams["font.sans-serif"] = ["Arial", "DejaVu Sans"]
mpl.rcParams["axes.titlesize"] = "x-large"  # medium, large, x-large, xx-large

mpl.style.use("seaborn-deep")

In [ ]:
occupancy_aruco = np.sum(occupancy_matrix_aruco.T == 1,axis=1)/(72000*50)
occupancy_sorted_aruco = -np.sort(-occupancy_aruco)
occupancy_cumsum_aruco = np.cumsum(occupancy_sorted_aruco)

occupancy_sleap = np.sum(occupancy_matrix_sleap.T == 1,axis=1)/(72000*50)
occupancy_sorted_sleap = -np.sort(-occupancy_sleap)
occupancy_cumsum_sleap = np.cumsum(occupancy_sorted_sleap)

occupancy_naps = np.sum(occupancy_matrix_naps.T == 1,axis=1)/(72000*50)
occupancy_sorted_naps = -np.sort(-occupancy_naps)
occupancy_cumsum_naps = np.cumsum(occupancy_sorted_naps)


t50_aruco = min(np.where(occupancy_cumsum_aruco > 0.5)[0])
t50_sleap = min(np.where(occupancy_cumsum_sleap > 0.5)[0])
t50_naps = min(np.where(occupancy_cumsum_naps > 0.5)[0])

In [ ]:
import palettable
# palette = sns.color_palette(cc.glasbey, n_colors=arr.shape[3])
fig, ax = plt.subplots(3, 1, figsize=(12, 9))
# fig.suptitle("Missingness frequency of tracklets")
sns.set_style("ticks")

data = pd.DataFrame({"x": np.arange(1,len(occupancy_cumsum_aruco)+1),"y":occupancy_cumsum_aruco})

sns.lineplot(
    ax=ax[0],
    x="x",
    y="y",
    data=data,
    color=palettable.wesanderson.Moonrise7_5.mpl_colors[0],
)
ax[0].fill_between(data['x'], data['y'], alpha=0.2,color=palettable.wesanderson.Moonrise7_5.mpl_colors[0])
ax[0].set(xlabel=None, ylabel=None)
ax[0].set_xlim(0,len(occupancy_cumsum_aruco))
ax[0].set_title("ArUco")
ax[0].axvline(x=t50_aruco, ymin=0,ymax=occupancy_cumsum_aruco[t50_aruco-1],color="black",linewidth=2,linestyle="--")
data = pd.DataFrame({"x": np.arange(1,len(occupancy_cumsum_sleap)+1),"y":occupancy_cumsum_sleap})
sns.lineplot(
    ax=ax[1],
    x="x",
    y="y",
    data=data,
    color=palettable.wesanderson.Moonrise7_5.mpl_colors[1],
)
ax[1].fill_between(data['x'], data['y'], alpha=0.2,color=palettable.wesanderson.Moonrise7_5.mpl_colors[1])
ax[1].set(xlabel=None, ylabel="Fraction of Total Possible Identifications")
ax[1].set_xlim(0,len(occupancy_cumsum_sleap))
ax[1].set_title("SLEAP")
ax[1].axvline(x=t50_sleap, ymin=0,ymax=occupancy_cumsum_sleap[t50_sleap-1],color="black",linewidth=2,linestyle="--")
data = pd.DataFrame({"x": np.arange(1,len(occupancy_cumsum_naps)+1),"y":occupancy_cumsum_naps})
sns.lineplot(
    ax=ax[2],
    x="x",
    y="y",
    data=data,
    color=palettable.wesanderson.Moonrise7_5.mpl_colors[2],
)
ax[2].fill_between(data['x'], data['y'], alpha=0.2,color=palettable.wesanderson.Moonrise7_5.mpl_colors[2])
ax[2].set(xlabel="Number of Tracks", ylabel=None)
ax[2].set_xlim(0,len(occupancy_cumsum_naps))
ax[2].set_title("NAPS")
ax[2].axvline(x=t50_naps, ymin=0,ymax=occupancy_cumsum_naps[t50_naps-1],color="black",linewidth=2,linestyle="--")
plt.setp(ax,ylim=(0,1))

# plt.xlim(0, 16)
sns.despine(offset=3, trim=False)
# plt.legend(labels=["ArUco", "SLEAP", "NAPS"], title = "Source of Tracks")
fig.tight_layout(rect=[0.05, 0.05, 1, 1])

plt.savefig("figures/t50_distribution.png", dpi=600)